# Raster Mosaic in Python
### Step 1:
Lets start by importing the libraryes needed

In [2]:
from rasterio.plot import show
from rasterio.merge import merge
import rasterio as rio
from pathlib import Path

### Step: 2:
Specify path to input directory of rasters and create output directory

In [14]:
# designate path to rasters
raster_in_path = Path('K:/ATUR\ELEVATION/3DEP_10m_1_3_arcsecond/')
out_path = Path('G:Data/Mogollon_Rim/Elevation/')
Path('mosaic').mkdir(parents=True, exist_ok=True)
output_path = 'mosaic/Mogollon_10m_elevation.tif'
# list files in input directory
list(raster_in_path.iterdir())



[WindowsPath('K:/ATUR/ELEVATION/3DEP_10m_1_3_arcsecond/USGS_13_n32w109_20130911.tif'),
 WindowsPath('K:/ATUR/ELEVATION/3DEP_10m_1_3_arcsecond/USGS_13_n32w110_20211229.tif'),
 WindowsPath('K:/ATUR/ELEVATION/3DEP_10m_1_3_arcsecond/USGS_13_n32w111_20240401.tif'),
 WindowsPath('K:/ATUR/ELEVATION/3DEP_10m_1_3_arcsecond/USGS_13_n32w112_20240401.tif'),
 WindowsPath('K:/ATUR/ELEVATION/3DEP_10m_1_3_arcsecond/USGS_13_n32w113_20210615.tif'),
 WindowsPath('K:/ATUR/ELEVATION/3DEP_10m_1_3_arcsecond/USGS_13_n32w114_20240401.tif'),
 WindowsPath('K:/ATUR/ELEVATION/3DEP_10m_1_3_arcsecond/USGS_13_n33w108_20130911.tif'),
 WindowsPath('K:/ATUR/ELEVATION/3DEP_10m_1_3_arcsecond/USGS_13_n33w109_20130911.tif'),
 WindowsPath('K:/ATUR/ELEVATION/3DEP_10m_1_3_arcsecond/USGS_13_n33w110_20220811.tif'),
 WindowsPath('K:/ATUR/ELEVATION/3DEP_10m_1_3_arcsecond/USGS_13_n33w111_20240401.tif'),
 WindowsPath('K:/ATUR/ELEVATION/3DEP_10m_1_3_arcsecond/USGS_13_n33w112_20240401.tif'),
 WindowsPath('K:/ATUR/ELEVATION/3DEP_10m_1_

In [10]:
# create a list from this directory
raster_files = list(raster_in_path.iterdir())

### Step 3
Lets create a little function so we can specify which files we want from this list

In [9]:
def filter_files_by_coordinate_pairs(file_list, coordinate_pairs):
    """
    Filters files by specified 'nxxwyyy' coordinate pairs.

    Args:
    file_list (list of Path): List of file paths.
    coordinate_pairs (list of str): List of 'nxxwyyy' values to include.

    Returns:
    list of Path: Filtered list of file paths.
    """
    filtered_files = []
    for file_path in file_list:
        filename = file_path.name
        # Check if filename contains any of the specified 'nxxwyyy' values
        if any(coord_pair in filename for coord_pair in coordinate_pairs):
            filtered_files.append(file_path)
    return filtered_files


In [15]:
# Example usage:
raster_in_path = Path("K:/ATUR/ELEVATION/3DEP_10m_1_3_arcsecond/")
file_list = list(raster_in_path.iterdir())

# Define the coordinate pairs you want to filter by
coordinate_pairs = ['n36w113', 'n34w111', 'n34w112', 'n35w110', 'n35w111', 'n35w112', 'n35w113']

# Filter files
selected_files = filter_files_by_coordinate_pairs(file_list, coordinate_pairs)
for file in selected_files:
    print(file)

K:\ATUR\ELEVATION\3DEP_10m_1_3_arcsecond\USGS_13_n34w111_20240402.tif
K:\ATUR\ELEVATION\3DEP_10m_1_3_arcsecond\USGS_13_n34w112_20240402.tif
K:\ATUR\ELEVATION\3DEP_10m_1_3_arcsecond\USGS_13_n35w110_20211207.tif
K:\ATUR\ELEVATION\3DEP_10m_1_3_arcsecond\USGS_13_n35w111_20240402.tif
K:\ATUR\ELEVATION\3DEP_10m_1_3_arcsecond\USGS_13_n35w112_20240402.tif
K:\ATUR\ELEVATION\3DEP_10m_1_3_arcsecond\USGS_13_n35w113_20240402.tif
K:\ATUR\ELEVATION\3DEP_10m_1_3_arcsecond\USGS_13_n36w113_20240402.tif


In [18]:
# create an empty list to populate
raster_to_mosaic = []


[]

### Step 4
loop through the selected files and open them, then append them to the raster

In [19]:
for p in selected_files:
    raster = rio.open(p)
    raster_to_mosaic.append(raster)
    

### Step 5
Merge the files into a mosaic

In [21]:
Mosaic, Output = merge(raster_to_mosaic)

RasterioIOError: Read or write failed. K:/ATUR/ELEVATION/3DEP_10m_1_3_arcsecond/USGS_13_n34w112_20240402.tif, band 1: IReadBlock failed at X offset 21, Y offset 4: TIFFReadEncodedTile() failed.

### Step 6
Next we copy the raster's metadata and update it to match the height and width of the mosaic

In [ ]:
output_meta = raster.meta.copy()
output_meta.update(
    {"driver": "GTiff",
        "height": Mosaic.shape[1],
        "width": Mosaic.shape[2],
        "transform": Output,
    }
)

### Step 7
Output the raster to the output folder


In [ ]:
with rio.open(output_path, “w”, **output_meta) as m:
    m.write(Mosaic)